## TOC:
* [First Bullet Header](#first-bullet)
* [Second Bullet Header](#second-bullet)
* [Limit by date](#Limit-by-date)

In [1]:
import weaviate
from datetime import datetime, timezone
from termcolor import colored


In [2]:
def pprint_response(response: dict) -> None:
    
    response = response["data"]["Get"]

    for class_name in response:
        print(colored(class_name.upper(), 'red', 'on_white', attrs=['bold']))

        for idx, resp in enumerate(response[class_name]):
            print(f"Response # {idx}")
            for key, value in resp.items():
                print(f"{colored(key, 'magenta', 'on_grey')}: {value}")
            print("=" * 120)

In [3]:
client = weaviate.Client("http://localhost:8080")


In [4]:
articles_count = client.query.aggregate(class_name="Article").with_meta_count().do()
authors_count = client.query.aggregate(class_name="Author").with_meta_count().do()
print(f"Articles: {articles_count}, Authors: {authors_count}\n")


Articles: {'data': {'Aggregate': {'Article': [{'meta': {'count': 593}}]}}}, Authors: {'data': {'Aggregate': {'Author': [{'meta': {'count': 187}}]}}}



In [5]:
nearText = {
    "concepts": ["fashion"],
    "certainty": 0.5,
    "moveAwayFrom": {
        "concepts": ["financial"],
        "force": 0.8,
    },
    "moveTo": {
        "concepts": ["couturier"],
        "force": 0.85,
    },
}
response = (
    client.query.get(
        "Article",
        [
            "title",
            "short_description",
            "description",
            "_additional {certainty}",
        ],
    )
    .with_near_text(nearText)
    .with_limit(1)
    .do()
)

# print(response["data"]["Get"]["Article"][0]["description"])
# prettify_response(response, field="description")
for idx, article in enumerate(response["data"]["Get"]["Article"]):
    print(f"Response # {idx}")
    print(f"Title: {article['title']}")
    print(f"Description: {article['description']}")


Response # 0
Title: Nadja Swarovski: The luxury sector has been ‘incredibly resilient’ amid the pandemic
Description: LONDON — Nadja Swarovski, chair of the Swarovski Foundation, said that the luxury sector had effectively embraced the challenge of operating and reaching consumers amid the coronavirus pandemic. Swarovski, the first female executive board member for her family's luxury jewelry brand, was speaking at CNBC's Evolve Global Summit on Wednesday. She told CNBC's Tania Bryer that the "luxury sector, to a certain extent has been incredibly resilient and I think those companies who have been digitally savvy have absolutely embraced this opportunity to communicate with their customers on a digital scale." In addition, Swarovski said it had been "fantastic to see how the fashion industry has embraced this dilemma," with non-essential retail businesses in many countries having been temporarily closed due to lockdowns and other Covid-19 public health restrictions for most of the pas

In [6]:
pprint_response(response)

ARTICLE
Response # 0
_additional: {'certainty': 0.645875}
description: LONDON — Nadja Swarovski, chair of the Swarovski Foundation, said that the luxury sector had effectively embraced the challenge of operating and reaching consumers amid the coronavirus pandemic. Swarovski, the first female executive board member for her family's luxury jewelry brand, was speaking at CNBC's Evolve Global Summit on Wednesday. She told CNBC's Tania Bryer that the "luxury sector, to a certain extent has been incredibly resilient and I think those companies who have been digitally savvy have absolutely embraced this opportunity to communicate with their customers on a digital scale." In addition, Swarovski said it had been "fantastic to see how the fashion industry has embraced this dilemma," with non-essential retail businesses in many countries having been temporarily closed due to lockdowns and other Covid-19 public health restrictions for most of the past year. Swarovski pointed to the use of digital

# Limit by date <a class="anchor" id="first-bullet"></a>

In [7]:
date = datetime(year=2020, month=1, day=1, tzinfo=timezone.utc).isoformat()
date


'2020-01-01T00:00:00+00:00'

In [8]:
nearText = {
    "concepts": ["car industry", "risks", "europe"],
    "certainty": 0.5,
}

response = (
    client.query.get(
        "Article",
        [
            "title",
            "short_description",
            "description",
            "published_at",
            "_additional {certainty}",
        ],
    )
    .with_where(
        {
            "operator": "LessThan",
            "path": ["published_at"],
            # valueDate: The date (ISO 8601 timestamp, formatted as RFC3339) value that the Path’s last property name should be compared to.
            "valueDate": date,
        }
    )
    .with_near_text(nearText)
    .with_limit(5)
    .do()
)

for idx, article in enumerate(response["data"]["Get"]["Article"]):
    print(f"Response # {idx}")
    print(f"Title: {article['title']}")
    print(f"Published: {article['published_at']}")
    print("=" * 100)


Response # 0
Title: Here are the top 10 major worries for global business leaders right now, according to WEF
Published: 2017-09-20T10:05:17Z
Response # 1
Title: European stocks close higher amid earnings, US jobs report; Wirecard plummets 25%
Published: 2019-02-01T07:02:21Z
Response # 2
Title: Forget the Brexit 'noise,' these are the real risks Europe faces
Published: 2016-09-09T09:17:00Z
Response # 3
Title: European stocks close lower as Italian banks plunge on political turmoil; Autos down 2.5%
Published: 2019-08-09T05:51:49Z
Response # 4
Title: European markets eke out gains at close as ECB holds interest rates; Akzo Nobel up 12.9%
Published: 2017-03-09T16:52:28Z


In [9]:
pprint_response(response)

ARTICLE
Response # 0
_additional: {'certainty': 0.71583223}
description: Cybersecurity, health of the global economy, energy price shock and terrorist attacks are some of the top risks that concern businesses and may threaten their ability to operate, according to results from a new survey by the World Economic Forum (WEF) published Wednesday.The survey, conducted annually by the WEF's strategic partners Marsh & McLennan Companies and Zurich Insurance Group, highlighted the following ten risks that businesses are presently concerned about:Unemployment and underemploymentFiscal crisesFailure of national governanceEnergy price shockProfound social instabilityFailure of financial mechanism or institutionFailure of critical infrastructureCyber attacksInter-state conflictTerrorist attacksWhile concerns about the overall health of the global economy continue to dominate the list, the report found concerns around cybersecurity risks rising in importance."Business leaders in many of world's la

# Limit by keywords <a class="anchor" id="second-bullet"></a>

In [10]:
nearText = {
    "concepts": ["car industry", "risks", "europe"],
    "certainty": 0.5,
}

response = (
    client.query.get(
        "Article",
        [
            "title",
            "keywords",
            "_additional {certainty}",
        ],
    )
    .with_where(
        {
            "operator": "Equal",
            "path": ["keywords"],
            # TODO: change to valueString after service restart
            "valueText": ["bonds", "fast money"],  # there are no articles with these keywords at the same time
            # "valueText": ["fast money"],
        }
    )
    .with_near_text(nearText)
    .with_limit(5)
    .do()
)

for idx, article in enumerate(response["data"]["Get"]["Article"]):
    print(f"Response # {idx}")
    print(f"Title: {article['title']}")
    print(f"Keywords: {article['keywords']}")
    print("=" * 100)


In [11]:
nearText = {
    "concepts": ["car industry", "risks", "europe"],
    "certainty": 0.5,
}

where_filter = {
    "operator": "Or",
    "operands": [
        {
            "operator": "Equal",
            "path": ["keywords"],
            "valueText": "bonds",
        },
        {
            "operator": "Equal",
            "path": ["keywords"],
            "valueText": "fast money",
        },
    ],
}

response = (
    client.query.get(
        "Article",
        [
            "title",
            "keywords",
            "_additional {certainty}",
        ],
    )
    .with_where(where_filter)
    .with_near_text(nearText)
    .with_limit(5)
    .do()
)

for idx, article in enumerate(response["data"]["Get"]["Article"]):
    print(f"Response # {idx}")
    print(f"Title: {article['title']}")
    print(f"Keywords: {article['keywords']}")
    print("=" * 100)


Response # 0
Title: Lasry: Europe could face Japan-style deflation era
Keywords: ['cnbc', 'Articles', 'World Markets', 'Bonds', 'Markets', 'Corporate bonds', 'Investment strategy', 'Corporate Debt', 'Investing', 'Credit and Debt', 'Squawk on the Street', 'Market Outlook', 'Europe Economy', 'source:tagname:CNBC US Source']
Response # 1
Title: EU, Iran to Impact Crude Oil Prices, Experts Say
Keywords: ['cnbc', 'Articles', 'CNBC TV', 'Fast Money Halftime Report', 'source:tagname:CNBC US Source']
Response # 2
Title: Sellers Worried about Europe, or Playing Range Game?
Keywords: ['cnbc', 'Articles', 'S&P 500 Index', 'Bank of America Corp', 'BHP Group Ltd', 'Freeport-McMoRan Inc', 'HP Inc', 'Intel Corp', 'Oracle Corp', 'US Bancorp', 'CBOE Volatility Index', 'United States Steel Corp', 'Industrial Select Sector SPDR Fund', "McDonald's Corp", 'CNBC TV', 'Fast Money', 'source:tagname:CNBC US Source']
Response # 3
Title: The Word On GM, J. Crew & Airline Puts
Keywords: ['cnbc', 'Articles', 'CNBC

In [12]:
nearText = {
    "concepts": ["car industry", "risks", "europe"],
    "certainty": 0.5,
}

where_filter = {
    "operator": "Equal",
    "path": ["keywords"],
    "valueText": ["fast"],
}


response = (
    client.query.get(
        "Article",
        [
            "title",
            "keywords",
            "_additional {certainty}",
        ],
    )
    .with_where(where_filter)
    .with_near_text(nearText)
    .with_limit(5)
    .do()
)

for idx, article in enumerate(response["data"]["Get"]["Article"]):
    print(f"Response # {idx}")
    print(f"Title: {article['title']}")
    print(f"Keywords: {article['keywords']}")
    print("=" * 100)


Response # 0
Title: EU, Iran to Impact Crude Oil Prices, Experts Say
Keywords: ['cnbc', 'Articles', 'CNBC TV', 'Fast Money Halftime Report', 'source:tagname:CNBC US Source']
Response # 1
Title: Sellers Worried about Europe, or Playing Range Game?
Keywords: ['cnbc', 'Articles', 'S&P 500 Index', 'Bank of America Corp', 'BHP Group Ltd', 'Freeport-McMoRan Inc', 'HP Inc', 'Intel Corp', 'Oracle Corp', 'US Bancorp', 'CBOE Volatility Index', 'United States Steel Corp', 'Industrial Select Sector SPDR Fund', "McDonald's Corp", 'CNBC TV', 'Fast Money', 'source:tagname:CNBC US Source']
Response # 2
Title: The Word On GM, J. Crew & Airline Puts
Keywords: ['cnbc', 'Articles', 'CNBC TV', 'Fast Money', 'source:tagname:CNBC US Source']
Response # 3
Title: Pick a Direction and Go With It!
Keywords: ['cnbc', 'Articles', 'CNBC TV', 'Fast Money', 'source:tagname:CNBC US Source']
Response # 4
Title: In The Red For '07
Keywords: ['cnbc', 'Articles', 'CNBC TV', 'Fast Money', 'source:tagname:CNBC US Source']


In [13]:
pprint_response(response)

ARTICLE
Response # 0
_additional: {'certainty': 0.65679705}
keywords: ['cnbc', 'Articles', 'CNBC TV', 'Fast Money Halftime Report', 'source:tagname:CNBC US Source']
title: EU, Iran to Impact Crude Oil Prices, Experts Say
Response # 1
_additional: {'certainty': 0.6142397}
keywords: ['cnbc', 'Articles', 'S&P 500 Index', 'Bank of America Corp', 'BHP Group Ltd', 'Freeport-McMoRan Inc', 'HP Inc', 'Intel Corp', 'Oracle Corp', 'US Bancorp', 'CBOE Volatility Index', 'United States Steel Corp', 'Industrial Select Sector SPDR Fund', "McDonald's Corp", 'CNBC TV', 'Fast Money', 'source:tagname:CNBC US Source']
title: Sellers Worried about Europe, or Playing Range Game?
Response # 2
_additional: {'certainty': 0.61110544}
keywords: ['cnbc', 'Articles', 'CNBC TV', 'Fast Money', 'source:tagname:CNBC US Source']
title: The Word On GM, J. Crew & Airline Puts
Response # 3
_additional: {'certainty': 0.60470885}
keywords: ['cnbc', 'Articles', 'CNBC TV', 'Fast Money', 'source:tagname:CNBC US Source']
title:

# Filter by minimum description length

In [14]:
nearText = {
    "concepts": ["car industry", "risks", "europe"],
    "certainty": 0.5,
}

response = (
    client.query.get(
        "Article",
        [
            "title",
            "keywords",
            "description",
            "_additional {certainty}",
        ],
    )
    .with_where(
        {
            "operator": "LessThan",
            "path": ["descriptionWordCount"],
            "valueInt": 100,
        }
    )
    .with_near_text(nearText)
    .with_limit(5)
    .do()
)

for idx, article in enumerate(response["data"]["Get"]["Article"]):
    print(f"Response # {idx}")
    print(f"Title: {article['title']}")
    print(f"Keywords: {article['keywords']}")
    print(f"Description: {article['description']}")
    print("=" * 100)


Response # 0
Title: Wells Fargo rates Ford, GM as overweight, praises new electric vehicles
Keywords: ['cnbc', 'Premium', 'Articles', 'Autos', 'Markets', 'Investment strategy', 'Ford Motor Co', 'General Motors Co', 'U.S. Markets Overview', 'Investing', 'PRO Home', 'CNBC Pro', 'Pro: Street Calls', 'source:tagname:CNBC US Source']
Description: Legacy automakers Ford and General Motors are set to be industry leaders once again as electric vehicles and autonomous driving come into focus, according Wells Fargo.The firm's analyst Colin Langan initiated coverage of both stocks on Monday with overweight ratings, praising the new and upcoming electric vehicles for the stalwart companies.
Response # 1
Title: Opening Calls for Europe's Major Indexes
Keywords: ['cnbc', 'Articles', 'Europe News', 'Business News', 'Economy', 'World Economy', 'source:tagname:CNBC US Source']
Description: More from CNBC.comEuropean Futures Rise after Fed CutLatest European Business News
Response # 2
Title: As it happe

In [15]:
pprint_response(response)

ARTICLE
Response # 0
_additional: {'certainty': 0.6467868}
description: Legacy automakers Ford and General Motors are set to be industry leaders once again as electric vehicles and autonomous driving come into focus, according Wells Fargo.The firm's analyst Colin Langan initiated coverage of both stocks on Monday with overweight ratings, praising the new and upcoming electric vehicles for the stalwart companies.
keywords: ['cnbc', 'Premium', 'Articles', 'Autos', 'Markets', 'Investment strategy', 'Ford Motor Co', 'General Motors Co', 'U.S. Markets Overview', 'Investing', 'PRO Home', 'CNBC Pro', 'Pro: Street Calls', 'source:tagname:CNBC US Source']
title: Wells Fargo rates Ford, GM as overweight, praises new electric vehicles
Response # 1
_additional: {'certainty': 0.6416493}
description: More from CNBC.comEuropean Futures Rise after Fed CutLatest European Business News
keywords: ['cnbc', 'Articles', 'Europe News', 'Business News', 'Economy', 'World Economy', 'source:tagname:CNBC US Sour

# Filter by title exact match

In [16]:
nearText = {
    "concepts": ["market prediction"],
    "certainty": 0.5,
}

where_filter = {
    "operator": "Equal",
    "path": ["title"],
    "valueText": ["google motorola"],
}

response = (
    client.query.get(
        "Article",
        [
            "title",
            "description",
            "_additional {certainty}",
        ],
    )
    .with_where(where_filter)
    .with_near_text(nearText)
    .with_limit(5)
    .do()
)

for idx, article in enumerate(response["data"]["Get"]["Article"]):
    print(f"Response # {idx}")
    print(f"Certainty: {article['_additional']['certainty']}")
    print(f"Title: {article['title']}")
    print(f"Description: {article['description']}")
    print("=" * 100)


Response # 0
Certainty: 0.556478
Title: Motorola Deal Buys Google Patent Protection: Schmidt
Description: Google's $12.5 billion acquisition of Motorola Mobilitygives the Internet search company patent protection while putting it squarely into the smartphone hardware business, Executive Chairman Eric Schmidt told CNBC Monday. But the acquisition won't affect how Google  does business with the other phone makers that use its Android operating system, including HTC, he added."We are going into the hardware business, but we’re going to keep it separate and we’re going to treat everybody else on a fair basis," he explained.Android has had more than 550,000 activations a day and is succeeding "precisely because there are so many partners," he said. "If we somehow p--- them off ... it would be a disaster to the Android strategy."Google is also taking advantage of Motorola Mobility's  17,000+ patents, a portfolio Schmidt called "second to none" at a time when Google faces "significant patent 

In [17]:
pprint_response(response)

ARTICLE
Response # 0
_additional: {'certainty': 0.556478}
description: Google's $12.5 billion acquisition of Motorola Mobilitygives the Internet search company patent protection while putting it squarely into the smartphone hardware business, Executive Chairman Eric Schmidt told CNBC Monday. But the acquisition won't affect how Google  does business with the other phone makers that use its Android operating system, including HTC, he added."We are going into the hardware business, but we’re going to keep it separate and we’re going to treat everybody else on a fair basis," he explained.Android has had more than 550,000 activations a day and is succeeding "precisely because there are so many partners," he said. "If we somehow p--- them off ... it would be a disaster to the Android strategy."Google is also taking advantage of Motorola Mobility's  17,000+ patents, a portfolio Schmidt called "second to none" at a time when Google faces "significant patent challenges from people who want to 

# Filter by title fuzzy match

Notes

Each query using the Like operator iterates over the entire inverted index for that property. The search time will go up linearly with the dataset size. Be aware that there might be a point where this query is too expensive and will not work anymore.

In [18]:
nearText = {
    "concepts": ["market"],
    "certainty": 0.1,
}

where_filter = {
    "operator": "Like",
    "path": ["title"],
    # TODO: test it with string property
    "valueText": ["morg*n"],
}

response = (
    client.query.get(
        "Article",
        [
            "title",
            "description",
            "_additional {certainty}",
        ],
    )
    .with_where(where_filter)
    .with_near_text(nearText)
    .with_limit(5)
    .do()
)

for idx, article in enumerate(response["data"]["Get"]["Article"]):
    print(f"Response # {idx}")
    print(f"Certainty: {article['_additional']['certainty']}")
    print(f"Title: {article['title']}")
    print(f"Description: {article['description']}")
    print("=" * 100)


Response # 0
Certainty: 0.54058105
Title: Morgan Stanley earnings, revenue top expectations
Description: Morgan Stanley's third-quarter revenue jumped  50 percent, helping adjusted earnings beat expectations, as  higher income from equities sales and trading made up for a  drop in the Wall Street bank and brokerage's fixed-income  business.  Morgan Stanley reported net income of $888 million, or 44 cents  per share, from continuing operations in the quarter. That  compared with a loss of $1 billion, or 55 cents per share, a  year earlier.  The year-earlier figure included a charge of $2.3 billion to  reflect a rise in the value of Morgan Stanley's debt.Excluding items, Morgan Stanley earned 50 cents per share,  beating the average analyst estimate of 40 cents per share,  according to Thomson Reuters I/B/E/S.  (Read more: Cashin: This reminds    me of the dot-com bubble)  Overall revenue rose to $7.93 billion, from $5.28 billion in the  same quarter last year, driven by equities trading

In [19]:
pprint_response(response)

ARTICLE
Response # 0
_additional: {'certainty': 0.54058105}
description: Morgan Stanley's third-quarter revenue jumped  50 percent, helping adjusted earnings beat expectations, as  higher income from equities sales and trading made up for a  drop in the Wall Street bank and brokerage's fixed-income  business.  Morgan Stanley reported net income of $888 million, or 44 cents  per share, from continuing operations in the quarter. That  compared with a loss of $1 billion, or 55 cents per share, a  year earlier.  The year-earlier figure included a charge of $2.3 billion to  reflect a rise in the value of Morgan Stanley's debt.Excluding items, Morgan Stanley earned 50 cents per share,  beating the average analyst estimate of 40 cents per share,  according to Thomson Reuters I/B/E/S.  (Read more: Cashin: This reminds    me of the dot-com bubble)  Overall revenue rose to $7.93 billion, from $5.28 billion in the  same quarter last year, driven by equities trading and the  company's fast-growing

# Also display authors

In [20]:
query = """
{
  Get {
    Article(where: {
        path: ["keywords"],    # Path to the property that should be used
        operator: Equal,  # operator
        valueText: "bonds"         # value (which is always = to the type of the path property)
      }, limit: 5) {
      title
      hasAuthors {
        ... on Author {
          name
        }
      }
    }
  }
}
"""

response = client.query.raw(query)
for idx, article in enumerate(response["data"]["Get"]["Article"]):
    print(f"Response # {idx}")
    # print(f"Certainty: {article['_additional']['certainty']}")
    print(f"Title: {article['title']}")
    # print("Authors:")
    # for author in article["hasAuthors"]:
    #     print(f"\t{author['name']}")
    authors = ",".join(author["name"] for author in article["hasAuthors"])
    print(f"Authors: {authors}")
    # print(f"Description: {article['description']}")
    print("=" * 100)

Response # 0
Title: Instead of shunning Saudi Arabia after Khashoggi killing, investors flock to $7.5 billion bond sale
Authors: Tom DiChristopher
Response # 1
Title: US Treasurys lower as investors focus on data, monitor Russia-US relations
Authors: Silvia Amaro
Response # 2
Title: 5 Stocks Insiders Love Right Now
Authors: Unknown
Response # 3
Title: 10-year Treasury yield falls to 0.8% as investors return to safety amid pause in stock rally
Authors: Elliot Smith
Response # 4
Title: Greenberg: Momentum King’s Next Big Move
Authors: Herb Greenberg


In [21]:
query = """
{
  Get {
    Article(where: {
        path: ["{placeholder}"],    # Path to the property that should be used
        operator: Equal,  # operator
        valueText: "bonds"         # value (which is always = to the type of the path property)
      }, limit: 5) {
      title
      hasAuthors {
        ... on Author {
          name
        }
      }
    }
  }
}
"""

import re

query = re.sub("{placeholder}", "keywords", query)
print(query)


{
  Get {
    Article(where: {
        path: ["keywords"],    # Path to the property that should be used
        operator: Equal,  # operator
        valueText: "bonds"         # value (which is always = to the type of the path property)
      }, limit: 5) {
      title
      hasAuthors {
        ... on Author {
          name
        }
      }
    }
  }
}



In [22]:
response = client.query.raw(query)
for idx, article in enumerate(response["data"]["Get"]["Article"]):
    print(f"Response # {idx}")
    # print(f"Certainty: {article['_additional']['certainty']}")
    print(f"Title: {article['title']}")
    # print("Authors:")
    # for author in article["hasAuthors"]:
    #     print(f"\t{author['name']}")
    authors = ",".join(author["name"] for author in article["hasAuthors"])
    print(f"Authors: {authors}")
    # print(f"Description: {article['description']}")
    print("=" * 100)

Response # 0
Title: Instead of shunning Saudi Arabia after Khashoggi killing, investors flock to $7.5 billion bond sale
Authors: Tom DiChristopher
Response # 1
Title: US Treasurys lower as investors focus on data, monitor Russia-US relations
Authors: Silvia Amaro
Response # 2
Title: 5 Stocks Insiders Love Right Now
Authors: Unknown
Response # 3
Title: 10-year Treasury yield falls to 0.8% as investors return to safety amid pause in stock rally
Authors: Elliot Smith
Response # 4
Title: Greenberg: Momentum King’s Next Big Move
Authors: Herb Greenberg


In [23]:
pprint_response(response)

ARTICLE
Response # 0
hasAuthors: [{'name': 'Tom DiChristopher'}]
title: Instead of shunning Saudi Arabia after Khashoggi killing, investors flock to $7.5 billion bond sale
Response # 1
hasAuthors: [{'name': 'Silvia Amaro'}]
title: US Treasurys lower as investors focus on data, monitor Russia-US relations
Response # 2
hasAuthors: [{'name': 'Unknown'}]
title: 5 Stocks Insiders Love Right Now
Response # 3
hasAuthors: [{'name': 'Elliot Smith'}]
title: 10-year Treasury yield falls to 0.8% as investors return to safety amid pause in stock rally
Response # 4
hasAuthors: [{'name': 'Herb Greenberg'}]
title: Greenberg: Momentum King’s Next Big Move


# Filter by number of wrote articles

In [24]:
# Find all articles that are written by authors that wrote at least two articles

get_articles_where = """
  {
    Get {
      Author(
        where:{
          valueInt: 2,
          operator: GreaterThanEqual,
          path: ["wroteArticles"]
        }, limit: 5
      ) {
        name
        wroteArticles {
          ... on Article {
            title
          }
        }
      }
    }
  }
"""

response = client.query.raw(get_articles_where)
# print(query_result)
for idx, article in enumerate(response["data"]["Get"]["Author"]):
    print(f"Response # {idx}")
    print(f"Author name: {article['name']}")
    titles = "\n\t".join(x["title"] for x in article["wroteArticles"])
    print(f"Titles: {titles}")

Response # 0
Author name: Leslie Josephs
Titles: Georgia's lieutenant governor says he will 'kill' Delta tax break unless airline reinstates relationship with NRA
	Raytheon and United Technologies agree to all-stock merger that would create aerospace behemoth
Response # 1
Author name: Tyler Clifford
Titles: Cramer adds new stocks, recommends buying 12 laggards in his Covid-19 index
	'I come to bury Bitcoin, not to praise it': UBS 
Response # 2
Author name: Sharon Epperson
Titles: Energy Falls Despite 'Above Average' Hurricane Forecast
	Want to start a business? Here's what you need to know
Response # 3
Author name: Pippa Stevens
Titles: Guggenheim says solar sell-off is a buying opportunity and has an unusual favorite stock
	Should CEO pay be tied to share price?
Response # 4
Author name: Chris Morris
Titles: Video Game Sales Get off to Slow Start in 2011
	 Did EA Bust the Social Gaming Bubble?


In [25]:
pprint_response(response)

AUTHOR
Response # 0
name: Leslie Josephs
wroteArticles: [{'title': "Georgia's lieutenant governor says he will 'kill' Delta tax break unless airline reinstates relationship with NRA"}, {'title': 'Raytheon and United Technologies agree to all-stock merger that would create aerospace behemoth'}]
Response # 1
name: Tyler Clifford
wroteArticles: [{'title': 'Cramer adds new stocks, recommends buying 12 laggards in his Covid-19 index'}, {'title': "'I come to bury Bitcoin, not to praise it': UBS\xa0"}]
Response # 2
name: Sharon Epperson
wroteArticles: [{'title': "Energy Falls Despite 'Above Average' Hurricane Forecast"}, {'title': "Want to start a business? Here's what you need to know"}]
Response # 3
name: Pippa Stevens
wroteArticles: [{'title': 'Guggenheim says solar sell-off is a buying opportunity and has an unusual favorite stock'}, {'title': 'Should CEO pay be tied to share price?'}]
Response # 4
name: Chris Morris
wroteArticles: [{'title': 'Video Game Sales Get off to Slow Start in 201

# Filter by author

In [26]:
where = """
{
  Get {
    Article(
      where:{
        valueString: "Tyler",
        operator: Equal,
        path: ["hasAuthors", "Author", "name"]
      }, limit: 5
    ) {
      title
      hasAuthors {
        ... on Author {
          name
        }
      }
    }
  }
 }
"""

response = client.query.raw(where)
# print(response)
for idx, article in enumerate(response["data"]["Get"]["Article"]):
    print(f"Response # {idx}")
    print(f"Title: {article['title']}")
    authors = ','.join(x["name"] for x in article["hasAuthors"])
    print(f"Author name: {authors}")
    print("=" * 100)


Response # 0
Title: Your first trade for Wednesday, January 15
Author name: Tyler Bailey
Response # 1
Title: Cramer adds new stocks, recommends buying 12 laggards in his Covid-19 index
Author name: Tyler Clifford
Response # 2
Title: 'I come to bury Bitcoin, not to praise it': UBS 
Author name: Tyler Clifford


In [27]:
pprint_response(response)

ARTICLE
Response # 0
hasAuthors: [{'name': 'Tyler Bailey'}]
title: Your first trade for Wednesday, January 15
Response # 1
hasAuthors: [{'name': 'Tyler Clifford'}]
title: Cramer adds new stocks, recommends buying 12 laggards in his Covid-19 index
Response # 2
hasAuthors: [{'name': 'Tyler Clifford'}]
title: 'I come to bury Bitcoin, not to praise it': UBS 


# All combined

In [28]:
query = """
{
  Get {
    Article(
      limit: 5
      nearText: 
      {
        concepts: ["covid affect on stock market in usa"],
        moveAwayFrom:
        {
          concepts: ["new york", "cuomo"],
          force: 0.1
        }
      }
      where: 
      {
        operator: And,
        operands: 
        [
          {
            operator: GreaterThan,
            path: ["published_at"],
            valueDate: "2015-01-01T0:00:00.52Z",
          },
          {
            operator: Equal,
            path: ["keywords"],
            valueText: "bonds",
          },
          {
            operator: Equal,
            path: ["title"],
            valueText: "Europe Japan",
          },
          {
            operator: Equal,
            path: ["hasAuthors", "Author", "name"],
            valueString: "Tom DiChristopher",
          },
          {
            operator: GreaterThan,
            path: ["descriptionWordCount"],
            valueInt: 100,
          }
        ]
      }
    ) 
    {
      title
      description
      keywords
      published_at
      descriptionWordCount
      hasAuthors {
        ... on Author {
          name
        }
      }
      _additional {
        certainty
      }
    }
  }
}
"""

response = client.query.raw(query)

print(response)

{'data': {'Get': {'Article': [{'_additional': {'certainty': 0.6688515}, 'description': 'Europe could be looking at a Japan-style deflationary environment  for the next five years, investor Marc Lasry told CNBC on  Wednesday. Read MoreEuro tests low last  seen at its birth in1999  Lasry\'s Avenue Capital is continuing to buy credit-side debt at a  discount in Europe. Over the last three or four years, the amount  of debt that European banks have sold has increased by 100  percent, he said in a "Squawk  Box" interview.   "The way that the banks were able to sell this debt is, they keep  on buying sovereign debt, and then through that they make their  profits, and then each year they end up using those profits to  offset losses on that. And that\'s sort of of what happened in  Japan over a 10 year period," said Lasry, who specializes in  distressed debt investments.  Read More Can cash-strapped Europe  prevent terrorism surge?  The way for Europe to avoid a deflationary period is to clock

In [29]:
pprint_response(response)

ARTICLE
Response # 0
_additional: {'certainty': 0.6688515}
description: Europe could be looking at a Japan-style deflationary environment  for the next five years, investor Marc Lasry told CNBC on  Wednesday. Read MoreEuro tests low last  seen at its birth in1999  Lasry's Avenue Capital is continuing to buy credit-side debt at a  discount in Europe. Over the last three or four years, the amount  of debt that European banks have sold has increased by 100  percent, he said in a "Squawk  Box" interview.   "The way that the banks were able to sell this debt is, they keep  on buying sovereign debt, and then through that they make their  profits, and then each year they end up using those profits to  offset losses on that. And that's sort of of what happened in  Japan over a 10 year period," said Lasry, who specializes in  distressed debt investments.  Read More Can cash-strapped Europe  prevent terrorism surge?  The way for Europe to avoid a deflationary period is to clock 4  to 5 percent G